# LLMs as Symbolic Pattern Machines

### Imports

In [6]:
import os
import sys
import random
import json
import time

In [7]:
sys.path.append("/media/hdd/usr/edo/egoProcel_mistakes")

In [8]:
from llama.generation import Llama

### Constants

In [9]:
JSONS_FOLDER = "/media/hdd/usr/edo/egoProcel_mistakes/data/mistake_jsons_split"
CORRECT_JSON_FOLDER = os.path.join(JSONS_FOLDER, "correct")
CORRECT_JSON_FILES = os.listdir(CORRECT_JSON_FOLDER)
MISTAKE_JSON_FOLDER = os.path.join(JSONS_FOLDER, "mistake")
MISTAKE_JSON_FILES = os.listdir(MISTAKE_JSON_FOLDER)

### Response function

In [16]:
class LLM:
    def __init__(
        self,
        ckpt_dir: str,
        tokenizer_path: str,
        max_seq_len: int = 512,
        max_batch_size: int = 6,
    ):
        self.generator = Llama.build(
            ckpt_dir=ckpt_dir,
            tokenizer_path=tokenizer_path,
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
        )

    def __call__(self, dialogs, max_gen_len=None, temperature=0.6, top_p=0.9):
        out = self.generator.chat_completion(
            dialogs,  # type: ignore
            max_gen_len=max_gen_len,
            temperature=temperature,
            top_p=top_p,
        )
        return out

In [17]:
llama = LLM(
    ckpt_dir="/media/ssd/usr/edo/llama/llama-2-7b-chat",
    tokenizer_path="/media/ssd/usr/edo/llama/tokenizer.model",
)

ValueError: Error initializing torch.distributed using env:// rendezvous: environment variable RANK expected, but not set

### Esempio di utilizzo

In [14]:
selected_json = CORRECT_JSON_FILES[random.randint(0, len(CORRECT_JSON_FILES))]

with open(os.path.join(JSONS_FOLDER, selected_json), "r") as f:
    curr_dict = json.load(f)
input_for_LLM = curr_dict["context_str"] + curr_dict["input_str"]
predicted = LLM(input_for_LLM)
print(input_for_LLM)
print("GT:\n", curr_dict["output_str"])
print("Predicted:\n", predicted[0])

nusar-2021_action_both_9026-c06e_9026_user_id_2021-02-03_170116.json


### Evaluation
For the time being, we cannot freely use the `openai API`, we are instead constrined on their rate limits:
- 3 PROMPTS/MIN
- 200 PROMPTS/DAY

This is why we use `time.sleep(60)` and break the evaluation at the 50® step.

#### Evaluate on all procedures

In [ ]:
tot = 0
correct = 0

for n, json_file in enumerate(ALL_MISTAKE_JSONS[:150]):
    tot += 1
    with open(os.path.join(JSONS_FOLDER, json_file), "r") as f:
        curr_dict = json.load(f)
    input_for_LLM = curr_dict["context_str"] + curr_dict["input_str"]
    predicted = LLM(input_for_LLM)
    predicted = predicted[0].strip()
    gt = curr_dict["output_str"].strip()
    print(
        "Procedure Label: {}\nGT: {}\nPred:{}\nCorrect: {}\n".format(
            curr_dict["procedure_label"], gt, predicted, gt == predicted
        )
    )
    if predicted == curr_dict["output_str"].strip():
        correct += 1

ratio = correct / tot
print("Ratio:", ratio, f"{correct}/{tot}")

#### Evaluate on correct procedures

In [ ]:
tot = 0
correct = 0

for n, json_file in enumerate(CORRECT_JSON_FILES):
    tot += 1
    print(json_file)
    with open(os.path.join(CORRECT_JSON_FOLDER, json_file), "r") as f:
        curr_dict = json.load(f)
    input_for_LLM = curr_dict["context_str"] + curr_dict["input_str"]
    predicted = LLM(input_for_LLM)
    predicted = predicted[0].strip()
    gt = curr_dict["output_str"].strip()
    print(
        "Procedure Label: {}\nGT: {}\nPred:{}\nCorrect: {}\n".format(
            curr_dict["procedure_label"], gt, predicted, gt == predicted
        )
    )
    if predicted == curr_dict["output_str"].strip():
        correct += 1

ratio = correct / tot
print("Ratio:", ratio, f"{correct}/{tot}")

#### Evaluate on mistaken procedures

In [ ]:
tot = 0
correct = 0

for n, json_file in enumerate(MISTAKE_JSON_FILES):
    tot += 1
    with open(os.path.join(MISTAKE_JSON_FOLDER, json_file), "r") as f:
        curr_dict = json.load(f)
    input_for_LLM = curr_dict["context_str"] + curr_dict["input_str"]
    predicted = LLM(input_for_LLM)
    predicted = predicted[0].strip()
    gt = curr_dict["output_str"].strip()
    print(
        "Procedure Label: {}\nGT: {}\nPred:{}\nCorrect: {}\n".format(
            curr_dict["procedure_label"], gt, predicted, gt == predicted
        )
    )
    if predicted == curr_dict["output_str"].strip():
        correct += 1

ratio = correct / tot
print("Ratio:", ratio, f"{correct}/{tot}")

### transformers GPT-2 
We experienced with GPT-2 from transformers, but the results are not consistent

In [ ]:
from transformers import pipeline, set_seed

model = "gpt2"  # 'gpt2-medium', 'gpt2-large', 'gpt2-xl'
max_token_produced = 100
num_returned_sequences = 1
tokenizer = GPT2Tokenizer.from_pretrained(model)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


def transformers_LLM(prompt):
    return generator(
        prompt,
        max_length=max_token_produced,
        num_return_sequences=num_returned_sequences,
    )[0]["generated_text"].replace(prompt, "")

In [ ]:
prompt = "input:\n 21, 29, 107, 141, 125\noutput:\n 143\n---\ninput:\n 143, 125, 141, 107, 29\noutput:\n 21\n---\ninput:\n 125, 143, 29, 21, 141\noutput:\n"
print(transformers_LLM(prompt))

## Split procedures

In [ ]:
def truncated_strings(json_fn):
    with open(os.path.join(JSONS_FOLDER, json_fn), "r") as f:
        curr_dict = json.load(f)
    all_truncated_prompts = []
    all_gts = []
    input_str = curr_dict["input_str"]
    context = curr_dict["context_str"]
    output_str = curr_dict["output_str"]
    input_prompt, sequence_, output_prompt, _ = curr_dict["input_str"].split("\n")
    sequence = sequence_.split(",")
    for i in range(len(sequence)):
        curr_str = (
            context
            + input_prompt
            + "\n"
            + ",".join(sequence[:i])
            + "\n"
            + output_prompt
            + "\n"
        )
        curr_res = sequence[i]
        all_truncated_prompts.append(curr_str)
        all_gts.append(curr_res)
    all_truncated_prompts.append(context + input_str)
    all_gts.append(output_str)
    return all_truncated_prompts, all_gts

#### Evaluate on correct procedures

In [ ]:
tot = 0
correct = 0

for n, json_file in enumerate(CORRECT_JSON_FILES):
    print(json_file)
    sequences, gts = truncated_strings(json_file)
    for input_str, gt in zip(sequences, gts):
        tot += 1
        # print("====\n", input_str)
        # print("GT:", gt)
        predicted = LLM(input_str)
        predicted = predicted[0].strip()
        gt = gt.strip()
        print("GT: {}\nPred:{}\nCorrect: {}\n".format(gt, predicted, gt == predicted))
        if predicted == gt:
            correct += 1

ratio = correct / tot
print("Ratio:", ratio, f"{correct}/{tot}")

In [ ]:
tot = 0
correct = 0
performance_dict_per_step = {}
for n, json_file in enumerate(MISTAKE_JSON_FILES):
    print(json_file)
    sequences, gts = truncated_strings(json_file)
    for input_str, gt in zip(sequences, gts):
        tot += 1
        # print("====\n", input_str)
        # print("GT:", gt)
        predicted = LLM(input_str)
        predicted = predicted[0].strip()
        gt = gt.strip()
        print("GT: {}\nPred:{}\nCorrect: {}\n".format(gt, predicted, gt == predicted))
        if predicted == gt:
            correct += 1

ratio = correct / tot
print("Ratio:", ratio, f"{correct}/{tot}")